In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pyopenlpt as lpt

redirector = lpt.PythonStreamRedirector()

In [2]:
config_file = "D:/0.Code/Test/SD0075/config.txt"
basic_settings = lpt.BasicSetting()
basic_settings.readConfig(config_file)

tr_cfg = lpt.TracerConfig()
tr_cfg.readConfig(basic_settings._object_config_paths[0], basic_settings)

Estimating OTF...
Camera 0 (a,b,c,alpha) = 124,0.952865,0.945061,0

Camera 1 (a,b,c,alpha) = 124,0.965964,0.957879,0

Camera 2 (a,b,c,alpha) = 124,0.953876,0.945466,0

Camera 3 (a,b,c,alpha) = 124,0.94073,0.954269,0



True

In [6]:
from scipy.io import loadmat

# check the ghost objs from the stereomatch step
track_path = "D:/0.Code/Test/SD0075/particles_75k_100f.mat"
tracks = loadmat(track_path)['tracks']  

pts_groundtrue = tracks[tracks[:, 3] == 1, :3].astype(np.float32)

obj3d_list = [lpt.Tracer3D(lpt.Pt3D(pt[0], pt[1], pt[2])) for pt in pts_groundtrue]
cam_list = basic_settings._cam_list
for obj in obj3d_list:
    obj.projectObject2D(cam_list)

# remove objs with one or more projection out of image bounds
w_img = cam_list[0].getNCol()
h_img = cam_list[0].getNRow()
obj3d_list_filtered = []
for obj in obj3d_list:
    valid = True
    for cam in range(len(cam_list)):
        if obj._obj2d_list[cam]._pt_center[0] < 0 or obj._obj2d_list[cam]._pt_center[0] >= w_img \
        or obj._obj2d_list[cam]._pt_center[1] < 0 or obj._obj2d_list[cam]._pt_center[1] >= h_img:
            valid = False
            break
    if valid:
        obj3d_list_filtered.append(obj)

obj3d_list = obj3d_list_filtered

obj2d_list = []
for cam in range(len(cam_list)):
    obj2ds = []
    for obj in obj3d_list:
        obj2ds.append(obj._obj2d_list[cam])
    obj2d_list.append(obj2ds)

print(f"Total {len(obj3d_list)} 3D objects with all projections in image bounds.")

Total 61832 3D objects with all projections in image bounds.


In [8]:
# test stereo matching
stereomatch = lpt.StereoMatch(cam_list, obj2d_list, tr_cfg)

with lpt.PythonStreamRedirector():
    obj3d_list = stereomatch.match()
print(f"Matched {len(obj3d_list)} 3D objects.")

# check ghost objs
import sys, pathlib
repo_root = pathlib.Path.cwd().parent          # for finding openLPT_utils
sys.path.insert(0, str(repo_root))

from openLPT_utils.ghost_eval import find_ghosts

pts_reconstructed = np.array(
    [[obj._pt_center[0], obj._pt_center[1], obj._pt_center[2]] for obj in obj3d_list], 
    dtype=np.float32)

tol_mm = 0.6 * .04
cmp_st = find_ghosts(pts_groundtrue, pts_reconstructed, tol_mm, use_ckdtree=True)

print(f"real={len(cmp_st['matched_indices'])}, ghosts={len(cmp_st['ghost_indices'])}")

Matched 61832 3D objects.
real=61832, ghosts=0
